In [0]:
%sql
CREATE TABLE IF NOT EXISTS odace.map.silver_gares (
  code_uic STRING COMMENT 'Code UIC',
  libelle STRING COMMENT 'Libellé',
  fret STRING COMMENT 'Indicateur fret',
  voyageurs STRING COMMENT 'Indicateur voyageurs',
  code_ligne STRING COMMENT 'Code de la ligne',
  rg_troncon INT COMMENT 'Rang du tronçon',
  pk STRING COMMENT 'Point kilométrique',
  commune STRING COMMENT 'Commune',
  departemen STRING COMMENT 'Département',
  idreseau STRING COMMENT 'Identifiant réseau',
  idgaia STRING COMMENT 'Identifiant GAIA',
  x_l93 DOUBLE COMMENT 'Coordonnée X (Lambert 93)',
  y_l93 DOUBLE COMMENT 'Coordonnée Y (Lambert 93)',
  longitude DOUBLE COMMENT 'Coordonnée X (WGS84)',
  latitude DOUBLE COMMENT 'Coordonnée Y (WGS84)',
  c_geo STRING COMMENT 'Code géographique',
  geo_point STRING COMMENT 'Point géographique',
  geo_shape STRING COMMENT 'Forme géographique',
  ingestion_timestamp TIMESTAMP COMMENT 'Horodatage d\'ingestion'
)
USING DELTA;

In [0]:
%sql
MERGE INTO odace.map.silver_gares AS target
USING (
    SELECT *
    FROM (
        SELECT *,
               ROW_NUMBER() OVER (PARTITION BY code_uic ORDER BY ingestion_timestamp DESC) AS rn
        FROM odace.map.bronze_gares
        WHERE voyageurs = 'O'
    ) sub
    WHERE rn = 1
) AS source
ON target.code_uic = source.code_uic
WHEN MATCHED THEN
    UPDATE SET
        target.libelle = source.libelle,
        target.fret = source.fret,
        target.voyageurs = source.voyageurs,
        target.code_ligne = source.code_ligne,
        target.rg_troncon = source.rg_troncon,
        target.pk = source.pk,
        target.commune = source.commune,
        target.departemen = source.departemen,
        target.idreseau = source.idreseau,
        target.idgaia = source.idgaia,
        target.x_l93 = source.x_l93,
        target.y_l93 = source.y_l93,
        target.longitude = source.x_wgs84,
        target.latitude = source.y_wgs84,
        target.c_geo = source.c_geo,
        target.geo_point = source.geo_point,
        target.geo_shape = source.geo_shape,
        target.ingestion_timestamp = source.ingestion_timestamp
WHEN NOT MATCHED THEN
    INSERT (
        code_uic, libelle, fret, voyageurs, code_ligne, rg_troncon, pk, commune, departemen, idreseau, idgaia, x_l93, y_l93, longitude, latitude, c_geo, geo_point, geo_shape, ingestion_timestamp
    )
    VALUES (
        source.code_uic, source.libelle, source.fret, source.voyageurs, source.code_ligne, source.rg_troncon, source.pk, source.commune, source.departemen, source.idreseau, source.idgaia, source.x_l93, source.y_l93, source.x_wgs84, source.y_wgs84, source.c_geo, source.geo_point, source.geo_shape, source.ingestion_timestamp
    );